In [25]:
def generate_latex_table(data):
    latex_header = r"""
\begin{table}[h]
    \centering
    \footnotesize
    \caption{Desktop con RTX3060}
    \begin{tabular}{@{}clccccccccccc@{}}
    \toprule
    & & \multicolumn{2}{c}{Latency {[}ms{]}} &  & Thr. {[}inf/s{]} &  \multicolumn{3}{c}{Model properties} &  & \multicolumn{2}{c}{Accuracy} \\ \cmidrule(lr){3-4}  \cmidrule(lr){7-9} \cmidrule(l){11-12} 
    & & ave.              & max.             &  &                       &  Size {[}MB{]}   & \# Layers   & \# Weights   &  & Top 1\%       & Top 5\%      \\ \midrule"""
    latex_footer = r"""
    \bottomrule
    \end{tabular}
\end{table}
    """

    latex_content = ""
    is_first = True
    for model_name, variants in data.items():
        if variants:
            model_section = f" "
            for i, variant in enumerate(variants):
                variant_name = variant['variant'].replace('_', ' ')  # Replace underscores with spaces
                if i == 0:
                    if is_first:
                        model_section += f" \\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{model_name}}}}}  "
                        model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                        is_first = False
                    else:
                        model_section += f" \\midrule \n"
                        model_section += f" \\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{model_name}}}}}  "
                        model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                   
                else:
                    model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                
                model_section += "\n"
            latex_content += model_section

    full_latex = latex_header + latex_content + latex_footer
    return full_latex


In [26]:
def parse_markdown_data(file_content):
    models_data = file_content.split('# ')[1:]  # Divide por modelo y batch size, ignorando el primer split que estaría vacío
    data_summary = {}
    inf_s_data = {}

    # Procesar y almacenar inf/s para batch size 256 por separado
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        if batch_size == 256:
            for line in lines:
                if '|' in line and 'Model' not in line and not all(char == '-' for char in line.replace('|', '').strip()):
                    cols = [col.strip() for col in line.split('|')]
                    if len(cols) > 1:
                        variant = cols[1]
                        inf_s = cols[2].split(' ')[0]
                        if model_name not in inf_s_data:
                            inf_s_data[model_name] = {}
                        inf_s_data[model_name][variant] = inf_s

    # Procesar datos principales para batch size 1 y añadir inf/s de batch size 256 cuando corresponda
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        if batch_size == 1:
            table_data = [line for line in lines if '|' in line and 'Model' not in line]

            data_summary[model_name] = []

            for data in table_data:
                if not all(char == '-' for char in data.replace('|', '').strip()):
                    cols = [col.strip() for col in data.split('|')]
                    if len(cols) > 1:
                        model_variant = cols[1]
                        inf_s = inf_s_data.get(model_name, {}).get(model_variant, 'N/A')  # Get inf/s from bs 256 data if available
                        lat_mean = cols[3].split(' / ')[0]
                        lat_worst = (cols[3].split(' / ')[1]).split(' ')[0]
                        lat_mean = lat_mean.strip()
                        lat_worst = lat_worst.strip()
                        size_mb = cols[4]
                        prec_1 = cols[5]
                        prec_5 = cols[6]
                        layers = cols[7]
                        params = cols[8]

                        model_data = {
                            'variant': model_variant,
                            'inf_s': inf_s,  # Added from bs 256 data
                            'lat_mean': lat_mean,
                            'lat_worst': lat_worst,
                            'size_mb': size_mb.strip(),
                            'prec_1': prec_1.strip(),
                            'prec_5': prec_5.strip(),
                            'layers': layers.strip(),
                            'params': params.strip()
                        }
                        data_summary[model_name].append(model_data)

    return data_summary


In [27]:
file_path = '../outputs/table_outputs/rtx3060/rtx3060_dynamic_batch.md'

# Abre y lee el archivo
with open(file_path, 'r', encoding='utf-8') as file:
    md_content = file.read()
# Parse the markdown content
parsed_data = parse_markdown_data(md_content)

print(parsed_data)

#for model_data in parsed_data.items():
#    for variants in model_data[1]:
#        print(model_data[0], variants)

{'mobilenet': [{'variant': 'Vanilla', 'inf_s': '1.305,6', 'lat_mean': '2.4', 'lat_worst': '4.9', 'size_mb': '13.6', 'prec_1': '72.02', 'prec_5': '90.63', 'layers': '104', 'params': '3487816'}, {'variant': 'TRT_fp32', 'inf_s': '2.670,0', 'lat_mean': '0.8', 'lat_worst': '5.0', 'size_mb': '14.2', 'prec_1': '72.02', 'prec_5': '90.62', 'layers': '57', 'params': '3469760'}, {'variant': 'TRT_fp16', 'inf_s': '3.529,2', 'lat_mean': '0.7', 'lat_worst': '1.8', 'size_mb': '8.5', 'prec_1': '71.98', 'prec_5': '90.63', 'layers': '58', 'params': '3469760'}, {'variant': 'TRT_int8', 'inf_s': '4.098,0', 'lat_mean': '0.6', 'lat_worst': '1.7', 'size_mb': '6.1', 'prec_1': '71.42', 'prec_5': '90.29', 'layers': '57', 'params': '3469760'}], 'resnet18': [{'variant': 'Vanilla', 'inf_s': '1.421,8', 'lat_mean': '1.7', 'lat_worst': '4.0', 'size_mb': '44.7', 'prec_1': '69.76', 'prec_5': '89.08', 'layers': '53', 'params': '11684712'}, {'variant': 'TRT_fp32', 'inf_s': '1.903,4', 'lat_mean': '1.3', 'lat_worst': '1.8', 

In [28]:
latex_table = generate_latex_table(parsed_data)
print(latex_table)


\begin{table}[h]
    \centering
    \footnotesize
    \caption{Desktop con RTX3060}
    \begin{tabular}{@{}clccccccccccc@{}}
    \toprule
    & & \multicolumn{2}{c}{Latency {[}ms{]}} &  & Thr. {[}inf/s{]} &  \multicolumn{3}{c}{Model properties} &  & \multicolumn{2}{c}{Accuracy} \\ \cmidrule(lr){3-4}  \cmidrule(lr){7-9} \cmidrule(l){11-12} 
    & & ave.              & max.             &  &                       &  Size {[}MB{]}   & \# Layers   & \# Weights   &  & Top 1\%       & Top 5\%      \\ \midrule  \multirow{4}{*}{\rotatebox[origin=c]{90}{mobilenet}}  & Vanilla & 2.4 & 4.9 & & 1.305,6 & 13.6 & 104 & 3487816 & & 72.02 & 90.63 \\
& TRT fp32 & 0.8 & 5.0 & & 2.670,0 & 14.2 & 57 & 3469760 & & 72.02 & 90.62 \\
& TRT fp16 & 0.7 & 1.8 & & 3.529,2 & 8.5 & 58 & 3469760 & & 71.98 & 90.63 \\
& TRT int8 & 0.6 & 1.7 & & 4.098,0 & 6.1 & 57 & 3469760 & & 71.42 & 90.29 \\
  \multirow{4}{*}{\rotatebox[origin=c]{90}{resnet18}}  & Vanilla & 1.7 & 4.0 & & 1.421,8 & 44.7 & 53 & 11684712 & & 69.76 & 89